In [98]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from langchain.agents import tool
from langchain.chains import LLMMathChain
from langchain.utilities import WikipediaAPIWrapper

from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Annotated
from langchain.schema import SystemMessage

import os


In [99]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [100]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=3, lang="en"))

@tool
def wiki_search(query: str) -> str:
    """Search Wikipedia for a given query."""
    try:
        result = wiki.run(query)
        if not result or "may refer to" in result.lower() or len(result.strip()) < 50:
            return "No relevant result found on Wikipedia."
        # Truncate very long results to keep memory manageable
        if len(result) > 1000:
            result = result[:1000] + "..."
        return result
    except Exception as e:
        return f"Wikipedia search failed: {str(e)}"

In [101]:
class AgentState(TypedDict):
    task: str
    memory: List[str]
    next_action: str
    failed_attempts: int


In [102]:
llm = ChatOpenAI(model="gpt-4.1-mini-2025-04-14", temperature=0)


In [103]:
def planner_node(state: AgentState) -> AgentState:
    # Count how much information we've gathered
    research_entries = [entry for entry in state['memory'] if entry.startswith("Researcher found:")]
    
    prompt = (
        f"You are planning how to complete this task: '{state['task']}'.\n"
        f"Here is what you've gathered so far:\n{state['memory']}\n\n"
        f"You have gathered {len(research_entries)} pieces of information.\n\n"
        "IMPORTANT: If you have gathered 3 or more pieces of research information, "
        "or if the information already covers the basic concepts of the task "
        "reply with exactly: 'STOP'\n\n"
        "If you need more specific information, provide a SHORT search term (1-3 words). "
        "Examples: 'photosynthesis', 'Leo Messi', 'shakira', 'american shorthair'"
    )
    response = llm.invoke(prompt)
    print(f"Planner: {response.content}")
    return {
        **state,
        "next_action": response.content,
        "memory": state["memory"] + [f"Planner: {response.content}"]
    }

def researcher_node(state: AgentState) -> AgentState:
    # Extract a clean search term from the planner's response
    search_term = state["next_action"].strip()
    
    # If it's too long or contains instructions, extract key terms
    if len(search_term.split()) > 3 or "should" in search_term.lower():
        # Look for key terms related to the task
        task_lower = state['task'].lower()
        
        # Extract potential search terms from the task
        important_words = []
        for word in task_lower.split():
            if len(word) > 3 and word not in ['what', 'how', 'does', 'work', 'works', 'tell', 'about']:
                important_words.append(word)
        
        # Use the first important word, or extract from search term
        if important_words:
            search_term = important_words[0]
        else:
            # Extract first meaningful word from search term
            words = [w for w in search_term.split() if len(w) > 3]
            search_term = words[0] if words else search_term.split()[0]
    
    print(f"Researcher: Searching for → '{search_term}'")
    result = wiki_search(search_term)
    
    if "No relevant result" in result:
        state["failed_attempts"] += 1
        print(f"❌ Search failed. Attempt {state['failed_attempts']}/3")
    else:
        state["failed_attempts"] = 0  # reset if successful
        print("✅ Found information!")

    return {
        **state,
        "memory": state["memory"] + [f"Researcher found: {result}"],
        "failed_attempts": state["failed_attempts"],
    }

def summarizer_node(state: AgentState) -> AgentState:
    # Extract only the research results for summarization
    research_entries = [entry for entry in state['memory'] if entry.startswith("Researcher found:")]
    research_data = "\n\n".join([entry.replace("Researcher found: ", "") for entry in research_entries])
    
    prompt = (
        f"Based on the research gathered, provide a concise and clear answer to: '{state['task']}'\n\n"
        f"Research data:\n{research_data}\n\n"
        "Please synthesize this information into a comprehensive but concise answer. "
        "Focus on the key concepts and processes. Keep it under 200 words."
    )
    
    response = llm.invoke(prompt)
    summary = response.content
    print(f"📝 Summarizer: {summary}")
    
    return {
        **state,
        "next_action": "COMPLETED",
        "memory": state["memory"] + [f"Final Summary: {summary}"]
    }

def should_continue(state: AgentState) -> str:
    # Check for explicit stop command
    if "stop" in state["next_action"].lower():
        print("🛑 Moving to summarizer...")
        return "summarize"
    
    # Count research entries
    research_entries = [entry for entry in state['memory'] if entry.startswith("Researcher found:")]
    
    # Auto-stop if we have enough information
    if len(research_entries) >= 3:
        print("🛑 Auto-stopping: Gathered sufficient information (3+ research results)")
        return "summarize"
    
    # Stop on too many failed attempts
    if state.get("failed_attempts", 0) >= 3:
        print("🔁 Too many failed attempts. Moving to summarizer...")
        return "summarize"
    
    # Prevent infinite loops with memory check
    if len(state["memory"]) > 12:  # Allow a bit more room
        print("🔄 Maximum iterations reached. Moving to summarizer...")
        return "summarize"
    
    return "research"

In [104]:
builder = StateGraph(AgentState)

builder.add_node("planner", planner_node)
builder.add_node("research", researcher_node)
builder.add_node("summarize", summarizer_node)

builder.set_entry_point("planner")
builder.add_edge("planner", "research")
builder.add_conditional_edges("research", should_continue, {
    "research": "planner",
    "summarize": "summarize"
})
builder.add_edge("summarize", END)

# Compile with a reasonable recursion limit
graph = builder.compile()

In [105]:

# Reset the state for a clean test
test_state = {
    "task": "Tell me about king abdallah",
    "memory": [],
    "next_action": "",
    "failed_attempts": 0
}

# Run with increased recursion limit but better auto-stopping
config = {"recursion_limit": 15}
try:
    final_state = graph.invoke(test_state, config=config)
    print("\n" + "=" * 60)
    print("✅ Execution completed successfully!")
    
    # Count different types of entries
    planner_entries = [e for e in final_state['memory'] if e.startswith("Planner:")]
    research_entries = [e for e in final_state['memory'] if e.startswith("Researcher found:")]
    summary_entries = [e for e in final_state['memory'] if e.startswith("Final Summary:")]
    
    print(f"📊 Statistics:")
    print(f"   • Total memory entries: {len(final_state['memory'])}")
    print(f"   • Planner decisions: {len(planner_entries)}")
    print(f"   • Research results: {len(research_entries)}")
    print(f"   • Final summaries: {len(summary_entries)}")
    print(f"   • Failed attempts: {final_state['failed_attempts']}")
    print(f"   • Status: {'✅ Completed with summary' if summary_entries else '⚠️ Incomplete'}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    
print("=" * 60)


Planner: King Abdallah
Researcher: Searching for → 'King Abdallah'
✅ Found information!
Planner: King Abdallah biography
Researcher: Searching for → 'King Abdallah biography'
✅ Found information!
Planner: King Abdullah II
Researcher: Searching for → 'King Abdullah II'
✅ Found information!
🛑 Auto-stopping: Gathered sufficient information (3+ research results)
📝 Summarizer: King Abdullah II of Jordan, born on January 30, 1962, is the reigning monarch of Jordan, having ascended the throne on February 7, 1999. He belongs to the Hashemite family, which has ruled Jordan since 1921, and is traditionally recognized as a 41st-generation direct descendant of the Prophet Muhammad. Abdullah is the eldest son of King Hussein and Princess Muna. Initially heir apparent, the title was transferred to his uncle Prince Hassan in 1965. Abdullah was educated both in Amman and abroad and began his military career in 1980. He rose through the ranks, commanding Jordan’s Special Forces in 1994 and becoming a m

In [106]:
# Display the conversation in a readable format
def display_conversation(state):
    print("🤖 AGENT CONVERSATION SUMMARY")
    print("=" * 60)
    print(f"📋 Task: {state['task']}")
    print(f"🔄 Failed Attempts: {state['failed_attempts']}")
    
    # Extract and display the final summary prominently
    final_summary = None
    for entry in state['memory']:
        if entry.startswith("Final Summary:"):
            final_summary = entry.replace("Final Summary: ", "")
            break
    
    if final_summary:
        print(f"\n🎯 FINAL ANSWER:")
        print("=" * 40)
        print(final_summary)
        print("=" * 40)
    
    print("\n📝 Detailed Conversation History:")
    print("-" * 40)
    
    for i, entry in enumerate(state['memory'], 1):
        if entry.startswith("Planner:"):
            print(f"\n{i}. 🧠 {entry}")
        elif entry.startswith("Researcher"):
            print(f"{i}. 🔍 {entry}")
        elif entry.startswith("Final Summary:"):
            print(f"{i}. 📝 ✨ {entry}")
        else:
            print(f"{i}. {entry}")
    
    print("\n" + "=" * 60)

# Display the results if the test ran successfully
try:
    if 'final_state' in locals():
        display_conversation(final_state)
except:
    print("Run the previous cell first to execute the graph!")


🤖 AGENT CONVERSATION SUMMARY
📋 Task: Tell me about king abdallah
🔄 Failed Attempts: 0

🎯 FINAL ANSWER:
King Abdullah II of Jordan, born on January 30, 1962, is the reigning monarch of Jordan, having ascended the throne on February 7, 1999. He belongs to the Hashemite family, which has ruled Jordan since 1921, and is traditionally recognized as a 41st-generation direct descendant of the Prophet Muhammad. Abdullah is the eldest son of King Hussein and Princess Muna. Initially heir apparent, the title was transferred to his uncle Prince Hassan in 1965. Abdullah was educated both in Amman and abroad and began his military career in 1980. He rose through the ranks, commanding Jordan’s Special Forces in 1994 and becoming a major general by 1998. In 1993, he married Rania Al-Yassin, and they have four children: Crown Prince Hussein, Princess Iman, Princess Salma, and Prince Hashem. Abdullah II’s reign continues the Hashemite legacy, emphasizing both Jordan’s stability and its historical linea

In [107]:
for entry in final_state["memory"]:
    print(entry + "\n" + "-" * 60)


Planner: King Abdallah
------------------------------------------------------------
Researcher found: Page: Abdullah II of Jordan
Summary: Abdullah II (Abdullah bin Hussein; born 30 January 1962) is King of Jordan, having ascended the throne on 7 February 1999. He is a member of the Hashemites, who have been the reigning royal family of Jordan since 1921, and is traditionally regarded a 41st-generation direct descendant of the prophet Muhammad.
Abdullah was born in Amman, as the first child of King Hussein and his wife, Princess Muna. As the king's eldest son, Abdullah was heir apparent until Hussein transferred the title to Abdullah's uncle Prince Hassan in 1965. Abdullah began his schooling in Amman, continuing his education abroad. He began his military career in 1980 as a training officer in the Jordanian Armed Forces, later assuming command of the country's Special Forces in 1994, eventually becoming a major general in 1998. In 1993, Abdullah married Rania Al-Yassin, with whom he 

In [108]:
# Quick function to extract and display just the final answer
def show_final_answer(state):
    print("🎯 QUICK ANSWER")
    print("=" * 50)
    print(f"Question: {state['task']}")
    print("-" * 50)
    
    for entry in state['memory']:
        if entry.startswith("Final Summary:"):
            answer = entry.replace("Final Summary: ", "")
            print(f"Answer: {answer}")
            break
    else:
        print("No final summary found. Run the full agent first!")
    
    print("=" * 50)

# Show the final answer if available
try:
    if 'final_state' in locals():
        show_final_answer(final_state)
    else:
        print("Run the test cell first to generate a final answer!")
except:
    print("Run the previous cell first to execute the graph!")


🎯 QUICK ANSWER
Question: Tell me about king abdallah
--------------------------------------------------
Answer: King Abdullah II of Jordan, born on January 30, 1962, is the reigning monarch of Jordan, having ascended the throne on February 7, 1999. He belongs to the Hashemite family, which has ruled Jordan since 1921, and is traditionally recognized as a 41st-generation direct descendant of the Prophet Muhammad. Abdullah is the eldest son of King Hussein and Princess Muna. Initially heir apparent, the title was transferred to his uncle Prince Hassan in 1965. Abdullah was educated both in Amman and abroad and began his military career in 1980. He rose through the ranks, commanding Jordan’s Special Forces in 1994 and becoming a major general by 1998. In 1993, he married Rania Al-Yassin, and they have four children: Crown Prince Hussein, Princess Iman, Princess Salma, and Prince Hashem. Abdullah II’s reign continues the Hashemite legacy, emphasizing both Jordan’s stability and its histori